In [1]:
from typing import Annotated
from typing_extensions import TypedDict

class AgentState(TypedDict):
    user_input: Annotated[str,lambda ole,new:new]
    email_template: Annotated[str, lambda old, new: new]
    user_feedback: Annotated[str,lambda old,new:new]
    approved: Annotated[bool,lambda old,new:new]

In [ ]:
from  langgraph.graph import StateGraph,START,END
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3-groq-tool-use")
graph_builder = StateGraph(AgentState)

def draft_content(state:AgentState):
    if(state['user_feedback']):
        prompt = f"""
        Here is the current email template:
        ---
        {state['email_template']}
        ---
        Update it based on this feedback:
        "{state['user_feedback']}"
        Only return the email content and nothing else.
        When you are updating the email, dont write here is the updated version.
        Return the email in json format with two keys one subject and the other content.
        """
    else:
         prompt = f"""
        Write a professional email based on this input:
        "{state['user_input']}"

        Use placeholders like {{name}}, {{email}}, {{company_name}} where needed.
        Only return the email content and nothing else.
        When you are updating the email, dont write here is the updated version.
        Return the email in json format with two keys one subject and the other content.
        """
    response = llm.invoke(prompt)
    print(response.content)
    return {"email_template": response.content}

In [21]:
def get_feedback(state:AgentState):
    print("\n📨 Draft Email:\n")
    print(state["email_template"])
    print("\n---")
    user_input = input("Approve this email? (yes or give feedback): ")
    if user_input.lower() in ["yes", "approve", "approved", "y"]:
        return {"approved": True}
    else:
        return {
            "user_feedback": user_input,
            "approved": False
        }

In [22]:
def send_emails(content:str):
    print("\n Emails with template \n")
    print(f"{content}")
    return {}

In [23]:
def route_tools(state:AgentState):
    if(state['approved']):
        return "send_emails"
    else:
        return "llm_writer"

In [24]:
from langgraph.graph import StateGraph,START,END

graph_builder = StateGraph(AgentState)
graph_builder.add_node("llm_writer",draft_content)
graph_builder.add_node("get_feedback",get_feedback)
graph_builder.add_node("send_emails",send_emails)

graph_builder.add_edge(START,"llm_writer")
graph_builder.add_edge("llm_writer","get_feedback")
graph_builder.add_conditional_edges("get_feedback",route_tools)
graph_builder.add_edge("send_emails",END)

graph = graph_builder.compile()

In [25]:
def run_agent(user_input: str):
    initial_state = {
        "user_input": user_input,
        "email_template": "",
        "user_feedback": "",
        "approved": False
    }

    final_state = graph.invoke(initial_state)
    print("\n✅ Agent Finished.")

In [26]:
run_agent("I want to send a welcome email to all the interns joining next week.")

Subject: Welcome Aboard! You're Now Part of the {company_name} Team.

Dear {name},

On behalf of everyone at {company_name}, I'm thrilled to welcome you aboard as an intern. Your enthusiasm and fresh perspectives are highly valued, and we're excited to have you join us next week!

If there's anything specific you need or any questions you have before your start date, please don't hesitate to reach out via email at {email}. We're here to help make your internship experience truly memorable.

Looking forward to seeing the amazing things you'll accomplish with us!

📨 Draft Email:

Subject: Welcome Aboard! You're Now Part of the {company_name} Team.

Dear {name},

On behalf of everyone at {company_name}, I'm thrilled to welcome you aboard as an intern. Your enthusiasm and fresh perspectives are highly valued, and we're excited to have you join us next week!

If there's anything specific you need or any questions you have before your start date, please don't hesitate to reach out via email 